In [107]:
!unzip -n train.zip

Archive:  train.zip


In [1]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

In [6]:
torch.cuda.set_device(0)
device = torch.device("cpu")

seed = 21
torch.cuda.manual_seed_all(seed)
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

In [7]:
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

In [8]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 64
train_val_ratio = 0.8

dataset = datasets.ImageFolder("data/train/", transform=transform)

classes = dataset.classes

train_size = int(train_val_ratio * len(dataset))
val_size = len(dataset) - train_size
train_set, val_set = torch.utils.data.random_split(dataset, [train_size, val_size])

train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=val_size, shuffle=False)

In [9]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        channels1 = 50
        channels2 = 100
        #channels3 = 64
        ## Warstwa konwolucyjna
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=channels1, kernel_size=3, stride=1, padding=0)
        ## Warstwa max pooling
        self.bn1 = nn.BatchNorm2d(channels1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.d1 = nn.Dropout2d()
        self.conv2 = nn.Conv2d(channels1, channels2, 3)
        self.bn2 = nn.BatchNorm2d(channels2)
        self.pool2 = nn.MaxPool2d(2)
        self.d2 = nn.Dropout2d()
        #self.conv3 = nn.Conv2d(channels2, channels3, 3)
        #self.bn3 = nn.BatchNorm2d(channels3)
        #self.pool3 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(channels2 * 14 * 14, 300)
        self.fc2 = nn.Linear(300, len(classes))
        self.drop = nn.Dropout()
        self.batch_norm1 = nn.BatchNorm1d(300)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.bn1(x)
        x = self.d1(x)
        x = self.pool2(F.relu(self.conv2(x)))
        x = self.bn2(x)
        x = self.d2(x)
        #x = self.pool3(F.relu(self.conv3(x)))
        #x = self.bn3(x)
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.batch_norm1(self.fc1(x)))
        x = self.drop(x)
        x = self.fc2(x)
        return x

net = Net().to(device)
net

Net(
  (conv1): Conv2d(3, 50, kernel_size=(3, 3), stride=(1, 1))
  (bn1): BatchNorm2d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (d1): Dropout2d(p=0.5, inplace=False)
  (conv2): Conv2d(50, 100, kernel_size=(3, 3), stride=(1, 1))
  (bn2): BatchNorm2d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (d2): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=19600, out_features=300, bias=True)
  (fc2): Linear(in_features=300, out_features=50, bias=True)
  (drop): Dropout(p=0.5, inplace=False)
  (batch_norm1): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [11]:
def validate():
    net.eval()
    with torch.no_grad():
        for x, labels in val_loader: # Val loader has only one batch with all data
            x, labels = x.to(device), labels.to(device)
            _, predictions = torch.max(net(x), 1)
            correct = predictions.eq(labels.view_as(predictions)).sum().item()
            return correct / len(x)

In [12]:
EPOCH_COUNT = 15

for epoch in range(EPOCH_COUNT):  # loop over the dataset multiple times
    running_loss = 0.0
    net.train()
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()

    print(f"[{epoch+1}/{EPOCH_COUNT}] loss: {running_loss / 1000:2.3f}, accuracy {validate() * 100:2.3f}%, const value {100.0 / len(classes):2.3f}%")
print('Finished Training')


[1/15] loss: 3.464, accuracy 29.228%, const value 2.000%
[2/15] loss: 3.142, accuracy 33.994%, const value 2.000%
[3/15] loss: 3.000, accuracy 36.801%, const value 2.000%
[4/15] loss: 2.884, accuracy 37.692%, const value 2.000%
[5/15] loss: 2.798, accuracy 38.976%, const value 2.000%
[6/15] loss: 2.728, accuracy 40.402%, const value 2.000%
[7/15] loss: 2.654, accuracy 41.118%, const value 2.000%
[8/15] loss: 2.585, accuracy 41.294%, const value 2.000%
[9/15] loss: 2.525, accuracy 42.124%, const value 2.000%
[10/15] loss: 2.471, accuracy 42.533%, const value 2.000%
[11/15] loss: 2.414, accuracy 42.782%, const value 2.000%
[12/15] loss: 2.365, accuracy 43.311%, const value 2.000%
[13/15] loss: 2.318, accuracy 43.010%, const value 2.000%
[14/15] loss: 2.270, accuracy 43.322%, const value 2.000%
[15/15] loss: 2.225, accuracy 43.322%, const value 2.000%
Finished Training


In [9]:
test_set = datasets.ImageFolder('data/test/', transform=transform)
filenames = [path.split('\\')[-1] for path, _ in test_set.imgs]

In [10]:
test_loader = torch.utils.data.DataLoader(test_set, batch_size=128, shuffle=False)
net.eval()
results = []
for data, _ in test_loader:
    outputs = net(data.to(device))
    _, preds = torch.max(outputs.data, 1)
    results.append(preds.numpy())
results = np.concatenate(results)

In [11]:
import pandas as pd

In [12]:
test_df = pd.DataFrame(data={'col1':filenames, 'col2':results})
test_df.to_csv('results.csv', header=None, index=False)